Installing HADOOP

In [16]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
!tar xf hadoop-3.3.2.tar.gz

import os
os.environ["HADOOP_CONF_DIR"] = "/content/hadoop-3.3.2/etc/hadoop"
!hadoop namenode -format
!hadoop --daemon start namenode
!hadoop --daemon start datanode
!jps
os.environ["PATH"] += os.pathsep + "$HADOOP_HOME/bin"

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
--2023-12-04 12:53:34--  https://archive.apache.org/dist/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638660563 (609M) [application/x-gzip]
Saving to: ‘hadoop-3.3.2.tar.gz.1’

hadoop-3.3.2.tar.gz 100%[===================>] 609.07M  15.2MB/s    in 51s     

2023-12-04 12:54:26 (11.9 MB/s) - ‘hadoop-3.3.2.tar.gz.1’ saved [638660563/638660563]

/bin/bash: hadoop: command not found
/bin/bash: hadoop: command not found
/bin/bash: hadoop: command not found
22503 Jps
14109 SparkSubmit


In [1]:
%pip install pyspark
%pip install findspark

import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
namenode_hostname = spark._jsc.hadoopConfiguration().get("fs.defaultFS").split("//")[1].split(":")[0]

print("Namenode hostname:", namenode_hostname)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 5.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 5.4 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=a611a7e112f94ee1f5cfdcd5129a446c87e227ef9aaec238ed7694330aebbb6d
  Stored in directory: /home/codespace/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


23/12/04 13:30:59 WARN Utils: Your hostname, codespaces-aff66d resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/12/04 13:30:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/04 13:31:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Namenode hostname: /


In [2]:
%pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 6.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.9 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [19]:
# Import necessary libraries
import os
import pandas as pd
import docx
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Read Word document using python-docx
file_path = "test.docx"
doc = docx.Document(file_path)

# Extract text from the Word document
paragraphs = doc.paragraphs
words = []
for paragraph in paragraphs:
    words += paragraph.text.split()

# Initialize Spark
spark = SparkSession.builder.appName("WordCount").getOrCreate()

# Convert words to RDD
words_rdd = spark.sparkContext.parallelize(words)

# Save words to local file if it does not exist
local_file_path = "/tmp/wordscount.txt"
if not os.path.exists(local_file_path):
    words_rdd.saveAsTextFile(local_file_path)

# Read words from local file
local_words = spark.sparkContext.textFile(local_file_path)

# Perform word counting
word_counts = local_words.flatMap(lambda line: line.split(" "))\
                         .filter(lambda word: isinstance(word, str))\
                         .map(lambda word: (word, 1))\
                         .reduceByKey(lambda a, b: a + b)

# Show the total word count
total_word_count = word_counts.collect()
total = 0
for (word, count) in total_word_count:
    total+=count
    print(word, count)
print("Total word count:", total)

cubilia 1
curae; 1
Pellentesque 1
lacinia 1
dictum 3
pharetra. 1
Vestibulum 2
lectus 1
magna 2
fermentum 3
felis 2
ornare, 1
quis 7
ex 4
Duis 4
porttitor 3
suscipit. 1
Aliquam 1
bibendum 1
leo, 1
condimentum 4
ante 3
scelerisque 2
quis. 1
Interdum 1
malesuada 4
ipsum 5
primis 2
in 4
dui. 1
amet 7
Nunc 2
eget 6
mi 2
ac, 3
feugiat 3
augue 5
at 3
Cras 5
rhoncus 1
ac. 2
varius, 1
ultricies, 1
metus 2
mattis 1
enim. 2
rutrum 4
sodales 2
fringilla. 1
condimentum. 1
Quisque 1
Sed 3
sed 5
sapien 4
faucibus 2
ornare 3
lorem, 1
vitae 7
varius 3
Vivamus 1
sapien. 1
Aenean 4
est 2
mollis 1
sollicitudin 3
nisi 2
tincidunt. 2
Proin 1
interdum 1
amet. 2
maximus 2
massa. 3
Maecenas 2
elit. 2
Lorem 1
consectetur 1
diam 1
porta, 1
facilisis. 1
commodo 2
sagittis 1
et. 2
lacinia. 1
nibh 3
finibus 1
felis. 1
rutrum, 1
quam 2
molestie 1
augue, 1
augue. 1
condimentum, 1
ut, 1
justo. 1
cursus 1
lacus 2
lorem. 1
pellentesque 2
dolor, 1
purus 1
vulputate, 1
Nam 3
nulla 2
feugiat. 2
ullamcorper 1
posuere. 1
ven